In [13]:
range_names = ["00 LJ vs BB 3Bet", "01 LJ vs BB Call", "02 LJ vs HJ 3Bet", "03 LJ vs CO 3Bet", "04 LJ vs BN 3Bet", "05 LJ vs SB 3Bet", "06 LJ vs HJ Call", "07 LJ vs CO Call", "08 LJ vs BN Call", "09 LJ vs SB Call", "10 HJ vs BB 3Bet", "11 HJ vs BB Call", "12 HJ vs CO 3Bet", "13 HJ vs BN 3Bet", "14 HJ vs SB 3Bet", "15 HJ vs CO Call", "16 HJ vs BN Call", "17 HJ vs SB Call", "18 CO vs BB 3Bet", "19 CO vs BB Call", "20 CO vs BN 3Bet", "21 CO vs SB 3Bet", "22 CO vs BN Call", "23 CO vs SB Call", "24 BN vs BB 3Bet", "25 BN vs BB Call", "26 BN vs SB 3Bet", "27 SB vs BB 3Bet", "28 SB vs BB Call", "29 BB 3Bet vs LJ", "30 BB Call vs LJ", "31 HJ 3Bet vs LJ", "32 CO 3Bet vs LJ", "33 BN 3Bet vs LJ", "34 SB 3Bet vs LJ", "35 HJ Call vs LJ", "36 CO Call vs LJ", "37 BN Call vs LJ", "38 SB Call vs LJ", "39 BB 3Bet vs HJ", "40 BB Call vs HJ", "41 CO 3Bet vs HJ", "42 BN 3Bet vs HJ", "43 SB 3Bet vs HJ", "44 CO Call vs HJ", "45 BN Call vs HJ", "46 SB Call vs HJ", "47 BB 3Bet vs CO", "48 BB Call vs CO", "49 BN 3Bet vs CO", "50 SB 3Bet vs CO", "51 BN Call vs CO", "52 SB Call vs CO", "53 BB 3Bet vs BN", "54 BB Call vs BN", "55 SB 3Bet vs BN", "56 BB 3Bet vs SB", "57 BB Call vs SB",]
range_name = range_names[34]
board_type = "two-tone"   # two-tone, rainbow, monotone


In [18]:
import os
import pandas as pd
from pandasql import sqldf
for range_name in range_names:
    df2 = None
    dfs = []
    for i in range(500):
        try:
            dfs.append(pd.read_csv("../reports/{}/{}/{}.csv".format(range_name, board_type, str(i).zfill(3))).drop(["Unnamed: 0"], axis=1))
        except:
            pass
    df = pd.concat(dfs, axis=0)
    df2 = sqldf("""select * from df a
            INNER JOIN (select flop, board_type, max(profit) AS max_profit from df group by 1,2) b on
            b.flop = a.flop and b.board_type = a.board_type and a.profit = b.max_profit
           order by 1 desc"""
          ,globals())
    try:
        os.mkdir("../reports/condensed_results/{}".format(board_type))
    except:
        pass
    df2.to_csv("../reports/condensed_results/{}/{}.csv".format(board_type, range_name))
    df2.head()




ValueError: No objects to concatenate

In [3]:
df2

,flop,board_type,0b,1b,2b,profit,flop,max_profit
0,"[9, 9, 8]",two-tone,16,11,10,1.333732,"[9, 9, 8]",1.333732
1,"[9, 9, 7]",two-tone,16,10,6,0.794836,"[9, 9, 7]",0.794836
2,"[9, 9, 6]",two-tone,16,11,8,0.626392,"[9, 9, 6]",0.626392
3,"[9, 9, 5]",two-tone,1,1,1,0.408262,"[9, 9, 5]",0.408262
4,"[9, 9, 4]",two-tone,1,0,0,0.024684,"[9, 9, 4]",0.024684
5,"[9, 9, 3]",two-tone,1,0,0,-0.003982,"[9, 9, 3]",-0.003982
6,"[9, 9, 2]",two-tone,1,0,0,-0.028538,"[9, 9, 2]",-0.028538
7,"[9, 8, 8]",two-tone,16,11,10,1.080869,"[9, 8, 8]",1.080869
8,"[9, 8, 7]",two-tone,2,2,2,0.395880,"[9, 8, 7]",0.395880
9,"[9, 8, 6]",two-tone,2,2,2,0.250720,"[9, 8, 6]",0.250720


([], [([12, 11], 4)], [([12, 11], 12)])